In [1]:
import dask.dataframe as dd
import pandas as pd
import os.path as op
import seaborn as sns

from glob import glob

/Users/richford/miniconda3/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
qc_files = glob("ratings/expertqc*.csv")

expert_qc = dd.read_csv(qc_files, usecols=["subject", "rating"], include_path_column=True).compute()
expert_qc["rater"] = expert_qc["path"].apply(lambda s: op.basename(s).replace("expertqc-", "").replace(".csv", ""))
expert_qc.drop("path", axis="columns", inplace=True)
expert_qc.reset_index(drop=True, inplace=True)

In [3]:
print(expert_qc.shape)

(800, 3)


In [4]:
expert_qc.head()

,subject,rating,rater
0,sub-NDARTF781TM8_ses-HBNsiteCBIC,1,wrathfulMuesli7
1,sub-NDARVC711ZK0_ses-HBNsiteCBIC,-1,wrathfulMuesli7
2,sub-NDARDX857DLB_ses-HBNsiteRU,1,wrathfulMuesli7
3,sub-NDARJH492TVW_ses-HBNsiteCBIC,1,wrathfulMuesli7
4,sub-NDARTB203DU7_ses-HBNsiteCBIC,-1,wrathfulMuesli7


In [5]:
expert_qc.groupby("subject").agg(["mean", "median", "std", "min", "max", "count"])

rating                               
                                   mean median       std min max count
subject                                                               
sub-NDARAC857HDB_ses-HBNsiteCBIC  -2.00   -2.0  0.000000  -2  -2     4
sub-NDARAH304ED7_ses-HBNsiteRU     0.50    0.5  1.290994  -1   2     4
sub-NDARAJ689BVN_ses-HBNsiteRU     1.00    1.0  0.816497   0   2     4
sub-NDARAP785CTE_ses-HBNsiteRU    -2.00   -2.0  0.000000  -2  -2     4
sub-NDARAU530GLJ_ses-HBNsiteCBIC   1.50    1.5  0.577350   1   2     4
...                                 ...    ...       ...  ..  ..   ...
sub-NDARZE963MEU_ses-HBNsiteCBIC   1.75    2.0  0.500000   1   2     4
sub-NDARZH672BAM_ses-HBNsiteRU     1.00    1.5  1.414214  -1   2     4
sub-NDARZR968TLL_ses-HBNsiteCUNY  -1.50   -1.5  0.577350  -2  -1     4
sub-NDARZV421TCZ_ses-HBNsiteCBIC  -1.50   -1.5  0.577350  -2  -1     4
sub-NDARZV749GAP_ses-HBNsiteCBIC   1.25    1.0  0.500000   1   2     4

[200 rows x 6 columns]